In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import os
import glob
import requests as rq
import wget
import lxml

# --proxy=https://webcache.comp.pge.com:8080

In [59]:
#locations of the outage downloads
path_download='/home/skotsan/Downloads/OUT/'
file_in="\\\\rcshare-nas05\\ElectricGasSupply\\Gas\Models_Tools\\Analysis_Models\\Intermittency_Project\\GasBurnModel\\Model_Enhance\\Inputs\\"
#Definition of functions to clean files

# Checks if the number is a float. Otherwise grabs digits before decimal point and after
def parse_numbers(elem):
    try:
        return float(elem)
    except:
        return float(elem.split('.')[0]+ "." + elem.split('.')[1])

# If the Header is loaded as string prepare the code that eliminates incorrect records    
def parse_string(elem):
    try:
        return float(elem)
    except:
        return 0.0  

In [48]:
#Generate Dates 30 days back + 7 Days into the future
numdays=120
base = dt.date.today()-dt.timedelta(days=numdays-1)
date_list = [base + dt.timedelta(days=x) for x in range(0, numdays)]
dates = pd.DataFrame(date_list, columns = ['Date'])
dates1=pd.to_datetime(dates['Date'])
dates['url']='http://content.caiso.com/unitstatus/data/unitstatus' + dates1.dt.strftime('%Y%m%d') + '0830.html'

epoch=len(dates)
dates['url'][0]

'http://content.caiso.com/unitstatus/data/unitstatus201810230830.html'

In [49]:
# Delete files in the directory prior to download
file_list=os.listdir(path_download)
for f in file_list:
    os.unlink(path_download + f)

In [50]:
# Download files
for j in range(epoch):
    file1=wget.download(dates['url'][j], out=path_download)

In [51]:
file_list0830=os.listdir(path_download)
len(file_list0830)

120

In [52]:
Outages_arr2=[]
epoch=len(file_list0830)
for j in range(epoch):
    Outages_new=pd.read_html(path_download + file_list0830[j], skiprows=1)[0]                     
    Outages_new.loc[-1]=Outages_new.columns
    Outages_new.index=Outages_new.index+1
    Outages_new.sort_index()
    Outages_new['Date']=dt.datetime.strptime(file_list0830[j][10:18], '%Y%m%d')
    Outages_arr2.extend(Outages_new.values)

In [60]:
# Capture files in the directory

Outages0830=pd.DataFrame(Outages_arr2)
Outages0830.columns=['Res_ID','Res_Name', 'Outage_Type', 'Capacity', 'Curtailed', 'Zone', 'Date' ]
Outages_clean=pd.DataFrame(index=Outages0830.index, columns=Outages0830.columns)
Outages_clean['Capacity']=Outages0830['Capacity'].apply(lambda x: parse_string(x)).astype(float)
Outages_clean['Curtailed']=Outages0830['Curtailed'].apply(lambda x: parse_string(x)).astype(float)
Outages_clean[['Res_ID','Res_Name', 'Outage_Type', 'Zone', 'Date' ]]=Outages0830[['Res_ID','Res_Name', 'Outage_Type', 'Zone', 'Date' ]]
Outages_clean['LAST_UPDATE']=dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [14]:
Out_curr = pd.read_csv( file_in + 'out.csv', parse_dates=['Date', 'LAST_UPDATE']).set_index('Date')
Out_curr.append(Outages_clean, sort=True)

,Capacity,Curtailed,Date,LAST_UPDATE,Outage,Outage_Type,Res_ID,Res_Name,Type,Zone
2014-01-01 00:00:00,22.00,NaN,NaN,2018-10-07 22:05:46,6.00,NaN,ADLIN_1_UNITS,GEYSERS AIDLIN AGGREGATE,Planned,ISO
2014-01-01 00:00:00,23.80,NaN,NaN,2018-10-07 22:05:46,23.80,NaN,ALMEGT_1_UNIT 1,ALAMEDA GT UNIT 1,Planned,NP15
2014-01-01 00:00:00,25.40,NaN,NaN,2018-10-07 22:05:46,25.40,NaN,ALMEGT_1_UNIT 2,ALAMEDA GT UNIT 2,Unplanned,NP15
2014-01-01 00:00:00,531.86,NaN,NaN,2018-10-07 22:05:46,73.86,NaN,APEX_2_MIRDYN,APEX_2_MIRDYN,Planned,ISO
2014-01-01 00:00:00,19.00,NaN,NaN,2018-10-07 22:05:46,2.00,NaN,AVENAL_6_SANDDG,Sand Drag Solar Project,Planned,ISO
2014-01-01 00:00:00,20.00,NaN,NaN,2018-10-07 22:05:46,2.00,NaN,AVENAL_6_SUNCTY,Sun City Solar Project,Planned,ISO
2014-01-01 00:00:00,230.00,NaN,NaN,2018-10-07 22:05:46,32.15,NaN,AVSOLR_2_SOLAR,AV SOLAR RANCH 1,Planned,ISO
2014-01-01 00:00:00,33.00,NaN,NaN,2018-10-07 22:05:46,1.20,NaN,BALCHS_7_UNIT 1,BALCH 1 PH UNIT 1,Unplanned,NP15
2014-01-01 00:00:00,428.00,NaN,NaN,2018-10-07 22:05:46,228.00,NaN,BCTSYS_5_PWXDYN,BCTSYS_5_PWXDYN,*Normal,ISO
2014-01-01 00:00:00,22.00,NaN,NaN,2018-10-07 22:05:46,9.00,NaN,BEARCN_2_UNITS,GEYSERS BEAR CANYON AGGREGATE,Planned,ISO


In [256]:
Out_curr.to_csv(file_in+'Outages_after2014.csv' )